In [4]:
## imports
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

ImportError: cannot import name 'Agent' from 'agents' (/Users/Hemant/Desktop/strique/seo-mcp-poc/src/agents/__init__.py)

In [9]:
load_dotenv(override=True)

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["DATAFORSEO_USERNAME"] = "hitesh.solanki@strique.io"
os.environ["DATAFORSEO_PASSWORD"] = "7836de8842bcdfc8"

In [10]:
# Generate GSC server
gsc_params = {
    "command": "/Users/Hemant/Desktop/strique/mcp-gsc/.venv/bin/python",
    "args": ["/Users/Hemant/Desktop/strique/mcp-gsc/gsc_server.py"],
    "env": {
        "GSC_CREDENTIALS_PATH": "/Users/Hemant/Downloads/service_account_credentials.json",
        "GSC_SKIP_OAUTH": "true"
    }   
}

async with MCPServerStdio(params=gsc_params, client_session_timeout_seconds=60) as server:
    gsc_tools = await server.list_tools()

gsc_tools

[Tool(name='list_properties', title=None, description="\n    Retrieves and returns the user's Search Console properties.\n    ", inputSchema={'properties': {}, 'title': 'list_propertiesArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'list_propertiesOutput', 'type': 'object'}, icons=None, annotations=None, meta=None),
 Tool(name='add_site', title=None, description='\n    Add a site to your Search Console properties.\n    \n    Args:\n        site_url: The URL of the site to add (must be exact match e.g. https://example.com, or https://www.example.com, or https://subdomain.example.com/path/, for domain properties use format: sc-domain:example.com)\n    ', inputSchema={'properties': {'site_url': {'title': 'Site Url', 'type': 'string'}}, 'required': ['site_url'], 'title': 'add_siteArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'requir

In [11]:
# Generate DataForSEO server
dataforseo_params = {
    "name": "DataForSEO",
    "description": "Access DataForSEO APIs",
    "transport": {
        "type": "stdio"
    },
    "command": "npx",
    "args": ["dataforseo-mcp-server"]
}

async with MCPServerStdio(params=dataforseo_params, client_session_timeout_seconds=60) as server:
    dataforseo_tools = await server.list_tools()

dataforseo_tools

[Tool(name='ai_optimization_keyword_data_locations_and_languages', title=None, description='Utility tool for ai_keyword_data_search_volume to get list of availible locations and languages', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=None, meta=None),
 Tool(name='ai_optimization_keyword_data_search_volume', title=None, description='This endpoint provides search volume data for your target keywords, reflecting their estimated usage in AI LLMs', inputSchema={'type': 'object', 'properties': {'keywords': {'type': 'array', 'items': {'type': 'string'}, 'description': 'Keywords. The maximum number of keywords you can specify: 1000'}, 'location_name': {'type': 'string', 'default': 'United States', 'description': "full name of the location, example: 'United Kingdom', 'United States'"}, 'language_code': {'type': 'string', 'description': "Search engine language cod

In [13]:
instructions = """
    You are a SEO expert. You are given a task to find the best keywords for a given website.
    You will use the following tools to accomplish your task:
    - DataForSEO
    - GSC   
"""


async with MCPServerStdio(params=dataforseo_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params= gsc_params, client_session_timeout_seconds=60) as mcp_server_browser:
        agent = Agent(
            name="seo-expert", 
            instructions=instructions, 
            model="gpt-4.1-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("seo-expert"):
            result = await Runner.run(agent, "Using DataForSEO, fetch the top 3 organic domains for best running shoes on Google India (google.co.in)")
            print(result.final_output)

The attempt to fetch the top organic domains for "best running shoes" on Google India using DataForSEO continues to fail with a fetch error. 

Is there an alternative approach or another method you would like me to try?


In [14]:
import nest_asyncio, asyncio, os
from agents import Agent, Runner
from agents.mcp import MCPServerStreamableHttp
from agents.model_settings import ModelSettings

# allow nested event loops in Jupyter
nest_asyncio.apply()

In [15]:
async def run_dataforseo_query():
    server = MCPServerStreamableHttp(
        name="dataforseo",
        params={
            "url": "http://localhost:3000/mcp",  # make sure server is running
            "timeout": 20,
        },
        cache_tools_list=True,
        max_retry_attempts=3,
    )

    async with server:
        agent = Agent(
            name="SEO Agent",
            instructions="Use the DataForSEO MCP tools to answer SEO questions.",
            mcp_servers=[server],
            model_settings=ModelSettings(tool_choice="auto"),
        )

        query = (
            "Get the top 3 Google result domains for 'iphone 16 price in india' "
            "from SERP (desktop, Google.in) and summarise briefly."
        )
        result = await Runner.run(agent, query)
        print("✅ Result:\n", result.final_output)

In [16]:
await run_dataforseo_query()

CancelledError: Cancelled via cancel scope 111ba3290